### Inspection of embedding space in presence of query and reference

In [1]:
# general libs
import numpy as np
import pandas as pd
import glob
import pickle
import torch

from random import sample

import re

# torch libs
import torch
import torch.nn as nn
import torch.nn.functional as F

# latent space libs
import umap
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler

# plotting libs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
from pandas import read_csv

# color palette
zeileis_28 = [
    "#023fa5",
    "#7d87b9",
    "#bec1d4",
    "#d6bcc0",
    "#bb7784",
    "#8e063b",
    "#4a6fe3",
    "#8595e1",
    "#b5bbe3",
    "#e6afb9",
    "#e07b91",
    "#d33f6a",
    "#11c638",
    "#8dd593",
    "#c6dec7",
    "#ead3c6",
    "#f0b98d",
    "#ef9708",
    "#0fcfc0",
    "#9cded6",
    "#d5eae7",
    "#f3e1eb",
    "#f6c4e1",
    "#f79cd4",
    # these last ones were added:
    '#7f7f7f',
    "#c7c7c7",
    "#1CE6FF",
    "#336600",
] 

In [ ]:
query_evidence = pd.read_csv('/stornext/General/data/academic/lab_davis/prot/benchmarking/PXD014777/HeLa_10_replicates_2hr/evidence.txt', 
                             sep='\t', engine='python', header=0)
pd.set_option('display.max_columns', None)


## data preprocessing
query_evidence = query_evidence.loc[(query_evidence['Intensity'] > 0) & \
                                (query_evidence['Charge'] != 1)]

# keep most intese peptide
query_evidence = query_evidence.loc[query_evidence.groupby(['Modified sequence', 'Charge', 'Raw file'])['Intensity'].idxmax()]


# remove contaminants
query_evidence = query_evidence.loc[[(not bool(re.search("CON__|REV__", i))) for i in query_evidence['Leading razor protein'].tolist()]]


query_evidence['study'] = 'query'
query_evidence['Species'] = 'HeLa'
query_evidence['PrecursorID'] = query_evidence['Modified sequence'].astype(str).str.cat(query_evidence.Charge.astype(str), sep='')


query_evidence


In [2]:
evidence_train = pd.read_pickle('PXD019086_PXD010012_combined_evidence_train_90Kto20Ksplit_5query_1shot_allPeptidesTxtFeatures_modSeqSpecies.pkl')

In [3]:
evidence_train

,Modified sequence,Sequence,Charge,Mass,m/z,Retention time,CCS,Gene names,Length,Raw file,...,Mass error [ppm],CCS length,Retention length,Ion mobility index,Ion mobility length,Number of isotopic peaks,Species,study,experiment_type,PrecursorID
1853774,_VISNPLLARK_,VISNPLLARK,2,1109.69208,555.853316,49.447,372.288031,RPS24B;RPS24A,10,20190304_tims03_FlMe_SA_200ng_Yeast_Lysc_IRT_F...,...,1.50620,6.307007,0.27169,615,16,3,Yeast,PXD019086,200ng_Yeast_Lysc_IRT_Fraction,_VISNPLLARK__2
1853775,_VISNPLLARK_,VISNPLLARK,2,1109.69208,555.853316,43.470,376.070427,RPS24B;RPS24A,10,20190304_tims03_FlMe_SA_200ng_Yeast_Lysc_IRT_F...,...,0.91928,89.609762,1.80470,606,214,5,Yeast,PXD019086,200ng_Yeast_Lysc_IRT_Fraction,_VISNPLLARK__2
1853776,_VISNPLLARK_,VISNPLLARK,2,1109.69208,555.853316,43.769,374.809613,RPS24B;RPS24A,10,20190304_tims03_FlMe_SA_200ng_Yeast_Lysc_IRT_F...,...,0.20340,42.836264,1.08670,609,103,5,Yeast,PXD019086,200ng_Yeast_Lysc_IRT_Fraction,_VISNPLLARK__2
1853781,_VISNPLLARK_,VISNPLLARK,2,1109.69208,555.853316,43.610,382.372619,RPS24B;RPS24A,10,20190304_tims03_FlMe_SA_200ng_Yeast_Lysc_IRT_F...,...,-0.16818,17.638467,0.65977,591,43,4,Yeast,PXD019086,200ng_Yeast_Lysc_IRT_Fraction,_VISNPLLARK__2
1853782,_VISNPLLARK_,VISNPLLARK,2,1109.69208,555.853316,43.374,378.591787,RPS24B;RPS24A,10,20190304_tims03_FlMe_SA_200ng_Yeast_Lysc_IRT_F...,...,0.12008,17.648187,0.62096,600,43,4,Yeast,PXD019086,200ng_Yeast_Lysc_IRT_Fraction,_VISNPLLARK__2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686997,_YYYVPADFVEYEK_,YYYVPADFVEYEK,2,1684.76609,843.390322,4159.100,NaN,PHKB,13,20180631_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_10...,...,3.32690,NaN,31.87200,446,31,5,HeLa,PXD010012,200ng_HeLa_50cm_100ms_120min_Fraction,_YYYVPADFVEYEK__2
1686998,_YYYVPADFVEYEK_,YYYVPADFVEYEK,2,1684.76609,843.390322,4131.700,NaN,PHKB,13,20180631_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_10...,...,3.71580,NaN,19.10000,432,28,5,HeLa,PXD010012,200ng_HeLa_50cm_100ms_120min_Fraction,_YYYVPADFVEYEK__2
1686999,_YYYVPADFVEYEK_,YYYVPADFVEYEK,2,1684.76609,843.390322,4126.700,NaN,PHKB,13,20180631_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_10...,...,5.06560,NaN,38.03200,437,49,6,HeLa,PXD010012,200ng_HeLa_50cm_100ms_120min_Fraction,_YYYVPADFVEYEK__2
669906,_YYYVPADFVEYEK_,YYYVPADFVEYEK,2,1684.76609,843.390322,92.273,449.532607,NaN,13,20190504_TIMS1_FlMe_SA_HeLa_frac10_B11_1_102,...,5.13220,18.487382,0.55038,426,46,6,HeLa,PXD019086,HeLa_frac10,_YYYVPADFVEYEK__2


In [ ]:
# append query to ref, shuffle rows, subsample and run umap

data = pd.concat([query_evidence, evidence_train], sort=False, ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
# high freq idents
pd.set_option('display.max_rows', None)
grouped = data.groupby(['PrecursorID', 'study'])
grouped.size()

In [4]:
input_dim = 7

class Encoder(nn.Module):

    def __init__(self, input_dim=8, hidden_dim=64, latent_dim=10):
        super().__init__()
        # Boring model
        self.encoder = nn.Sequential(
                    nn.Linear(input_dim, hidden_dim),
                    nn.ReLU()
                    # nn.Linear(in_size, out_size)
                    # nn.ReLU()
            )

        self.mean_encoder = nn.Linear(hidden_dim, latent_dim)
        self.var_encoder = nn.Linear(hidden_dim, latent_dim)


    def forward(self, x):
        # Simple forward
        hidden = self.encoder(x)
        mu = self.mean_encoder(hidden)
        logvar = self.var_encoder(hidden)

        # sample from the distribution having latent parameters z_mu and z_var
        # reparametrization
        # std = torch.exp(0.5*logvar)
        
        std = logvar
        # std = torch.exp(logvar) + 1e-4
        eps = torch.randn_like(std)
#         x_sample = mu + eps*std
        x_sample = mu

        # return x.view(x.size(0), -1)
        return x_sample
        

In [5]:
model = torch.load('../peptideprotonet/PXD019086_PXD010012_combined_evidence_90Kto20Ksplit_5query_1shot_fullmodel_featuresScaled_allPeptidesTxtFeatures_modSeqSpecies_hidden64_latent10_maxEpoch300_164trainways_xlatent_conditionalEmbedding.pth' ,
                          map_location=torch.device('cpu'))
model.eval()

Encoder(
  (encoder): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU()
  )
  (mean_encoder): Linear(in_features=64, out_features=10, bias=True)
  (var_encoder): Linear(in_features=64, out_features=10, bias=True)
)

In [6]:
# device = 'cuda'
device = 'cpu'
attr_names = ['Charge','Mass', 'm/z', 'Retention time','Retention length',
              'Ion mobility index', 'Ion mobility length','Number of isotopic peaks']


In [ ]:
x2 = torch.from_numpy(data[attr_names].to_numpy()).float()
x2 = StandardScaler().fit_transform(x2)
print(x2.shape)
z_query = model(torch.from_numpy(x2).float().to(device))
print(z_query.shape)

In [ ]:
protonet_embedding = z_query.cpu().detach().numpy()
print(protonet_embedding.shape)


In [ ]:
random.seed(64)
row_idx = np.random.choice(protonet_embedding.shape[0], 200000, replace=False)
z_subsample = protonet_embedding[row_idx]

In [ ]:
reducer = umap.UMAP(metric = 'cosine')
embedding = reducer.fit_transform(z_subsample)

In [ ]:
color_dict = {'PXD019086':'tab:blue', 'PXD010012':'tab:orange', 'query':'tab:green'}
markers = [plt.Line2D([0], [0], color=c, linestyle='', marker='o') for c in color_dict.values()]
labels = color_dict.keys()


plt.scatter(embedding[:,0], embedding[:,1],
           c=data.study[row_idx].map(color_dict), s = 0.1)
#plt.gca().set_aspect('equal', 'datalim')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.legend(markers, labels, bbox_to_anchor=(1.05, 1), loc='upper left')        
plt.title('peptideprotonet latent space')

In [ ]:
# color by charge - purple is charge 2
charge_state = data.Charge.astype("category")
charge_state = charge_state.to_numpy()
charge_sub = charge_state[row_idx]


plt.scatter(embedding[:,0], embedding[:,1],
           c=charge_sub, s = 0.1)
#plt.gca().set_aspect('equal', 'datalim')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.colorbar()
plt.title('peptideprotonet latent space')

In [ ]:
# evidence_train.reset_index(inplace=True, drop=True)

plt.scatter(embedding[:,0], embedding[:,1],
           c=data.CCS[row_idx], s = 0.1)
#plt.gca().set_aspect('equal', 'datalim')
plt.legend()
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.colorbar()
plt.title('peptideprotonet latent space\nCCS')

In [ ]:
plt.scatter(embedding[:,0], embedding[:,1],
           c=data['Retention time'][row_idx], s = 0.1)
#plt.gca().set_aspect('equal', 'datalim')
plt.legend()
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.colorbar()
plt.title('peptideprotonet latent space\nRT')
# plt.savefig('latent_twoStudy_conditionalModel_byRT.png')

In [ ]:
charge_state = data.Charge.astype("category")
charge_state = charge_state.to_numpy()
charge_sub = charge_state[row_idx][data.study[row_idx].isin(['query'])]


plt.scatter(embedding[data.study[row_idx].isin(['query']),0],
            embedding[data.study[row_idx].isin(['query']),1],
           c=charge_sub, s = 0.1)
#plt.gca().set_aspect('equal', 'datalim')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.colorbar()
plt.title('peptideprotonet latent space')

In [ ]:
# query = ['_IYNSIYIGSQDALIAHYPR__2']
#query = ['_KAEEEHLGILGPQLHADVGDK__3']
query = ['_KPADDQDPIDALSGDLDSCPSTTETSQNTAK__3']


modified_seq = data.PrecursorID
query_pep = np.zeros(len(modified_seq))
query_pep[modified_seq.isin(query)] = 1
query_pep = query_pep.astype('int')

xs = embedding[query_pep[row_idx]==1,0]
ys = embedding[query_pep[row_idx]==1,1]


plt.scatter(embedding[:,0], embedding[:,1],
           c='lavender', s = 0.4)


plt.scatter(xs, ys,
           c= data.study[row_idx][query_pep[row_idx] ==1].map(color_dict),
            s = 10.1)
#plt.gca().set_aspect('equal', 'datalim')
plt.legend()
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.title('peptideprotonet latent space\n _KPADDQDPIDALSGDLDSCPSTTETSQNTAK__3')
# plt.savefig('latent_twoStudy_conditionalModel_peptide2.png', bbox_inches='tight')

### Working with allPeptides table

#### Embedding query runs only

In [7]:
query_allpeptides = pd.read_csv('/stornext/General/data/academic/lab_davis/prot/benchmarking/PXD010012_PASEFF/HeLa_200ng_100ms_txt/txt/allPeptides.txt', sep='\t', engine='python', header=0)

In [8]:
query_allpeptides['study'] = 'query'
query_allpeptides['Species'] = 'HeLa'

In [9]:
query_allpeptides.head(10)

,Raw file,Charge,m/z,Mass,Resolution,Number of data points,Number of frames,Number of isotopic peaks,Isotope correlation,Mass fractional part,...,Ion mobility index,Ion mobility index length,Ion mobility index length (FWHM),Intensity,Intensities,Number of pasef MS/MS,Pasef MS/MS IDs,MS/MS scan number,study,Species
0,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,351.19328,350.18600,22164.869177,352,17,2,0.981385,0.186003,...,474.452,6,6,2621.8,NaN,0,NaN,NaN,query,HeLa
1,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,352.33801,351.33073,22539.827648,3302,20,3,0.999907,0.330732,...,530.617,36,21,57726.0,NaN,3,605922;605926;605930,-1.0,query,HeLa
2,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,338.34070,337.33342,22434.300059,1722,20,2,0.999209,0.333419,...,522.643,30,24,16680.0,NaN,5,615087;615094;615102;615110;615118,127.0,query,HeLa
3,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,352.33808,351.33081,22899.531531,7530,45,2,0.994022,0.330805,...,528.276,30,18,57695.0,NaN,6,612879;612884;612892;612900;612908;612917,175.0,query,HeLa
4,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,352.33805,351.33077,22830.932620,4893,39,2,0.998601,0.330769,...,531.098,33,18,50493.0,NaN,0,NaN,NaN,query,HeLa
5,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,352.33792,351.33065,22733.772428,4616,47,2,0.985606,0.330645,...,528.624,27,24,40727.0,NaN,7,610387;610395;610403;610411;610418;611788;611794,-1.0,query,HeLa
6,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,352.33796,351.33068,22623.992886,7049,42,2,0.998554,0.330680,...,530.211,33,18,70484.0,NaN,4,617142;617149;617156;617163,173.0,query,HeLa
7,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,352.33809,351.33082,22864.535214,13245,75,2,0.997476,0.330816,...,531.200,30,18,93649.0,NaN,3,607677;607683;607689,-1.0,query,HeLa
8,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,352.33787,351.33059,22864.520560,2165,20,2,0.998057,0.330590,...,529.966,33,18,24014.0,NaN,8,605021;605027;605032;605085;605091;605097;6051...,-1.0,query,HeLa
9,20180819_TIMS2_12-2_AnBr_SA_200ng_HeLa_50cm_12...,1,338.34089,337.33361,22823.026468,2877,32,2,0.993196,0.333612,...,534.201,45,27,39566.0,NaN,7,614048;614055;614062;614069;614075;614080;614085,129.0,query,HeLa


In [ ]:
# query_allpeptides = query_allpeptides.loc[(query_allpeptides['Intensity'] > 0) & \
#                                 (query_allpeptides['Charge'] != 1) &\
#                                 (query_allpeptides['Isotope correlation'] > 0.5) ]

In [10]:
query_allpeptides.shape

(1653858, 26)

In [11]:
attr_names = ['Charge','Mass', 'm/z', 'Retention time','Retention length',
              'Ion mobility index', 'Ion mobility index length','Number of isotopic peaks']
x = torch.from_numpy(query_allpeptides[attr_names].to_numpy()).float()
x = StandardScaler().fit_transform(x)
print(x.shape)
z = model(torch.from_numpy(x).float().to(device))

(1653858, 8)


In [12]:
protonet_embedding_z = z.cpu().detach().numpy()
print(protonet_embedding_z.shape)

(1653858, 10)


In [13]:
# append all meta-data
latent_dim = 10
latent_space = pd.DataFrame(protonet_embedding_z, columns = ['dim_'+str(i) for i in range(latent_dim)])


In [14]:
latent_space = pd.concat([latent_space, query_allpeptides], axis = 1)
latent_space.head(15)

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,Ion mobility index,Ion mobility index length,Ion mobility index length (FWHM),Intensity,Intensities,Number of pasef MS/MS,Pasef MS/MS IDs,MS/MS scan number,study,Species
0,-129.197174,-117.208076,98.712524,66.724258,-133.567673,29.731527,100.469551,-113.596436,-124.262711,-108.304062,...,474.452,6,6,2621.8,NaN,0,NaN,NaN,query,HeLa
1,-128.817245,-117.108658,98.461708,65.926186,-133.588394,29.718010,101.025475,-113.107025,-123.209831,-107.728905,...,530.617,36,21,57726.0,NaN,3,605922;605926;605930,-1.0,query,HeLa
2,-131.775375,-119.771523,100.874382,67.503334,-136.770523,30.658285,103.299866,-115.569244,-126.045509,-110.320335,...,522.643,30,24,16680.0,NaN,5,615087;615094;615102;615110;615118,127.0,query,HeLa
3,-129.272781,-118.125946,98.906441,66.217117,-133.691284,30.937622,101.164246,-113.181801,-124.168571,-108.477005,...,528.276,30,18,57695.0,NaN,6,612879;612884;612892;612900;612908;612917,175.0,query,HeLa
4,-129.192520,-117.848442,98.901360,66.096336,-133.762024,30.666540,101.215942,-113.061623,-123.886971,-108.324265,...,531.098,33,18,50493.0,NaN,0,NaN,NaN,query,HeLa
5,-129.306686,-118.265594,98.845596,66.334793,-133.606781,31.023409,101.062569,-113.286171,-124.365585,-108.559982,...,528.624,27,24,40727.0,NaN,7,610387;610395;610403;610411;610418;611788;611794,-1.0,query,HeLa
6,-129.231628,-117.956039,98.950928,66.115784,-133.756332,30.799273,101.234077,-113.086716,-123.992630,-108.401833,...,530.211,33,18,70484.0,NaN,4,617142;617149;617156;617163,173.0,query,HeLa
7,-129.832642,-119.565651,99.171890,66.592857,-133.564011,32.375969,101.274849,-113.684013,-125.495750,-109.284828,...,531.200,30,18,93649.0,NaN,3,607677;607683;607689,-1.0,query,HeLa
8,-128.852219,-117.181320,98.522141,66.061501,-133.756760,29.995850,101.041161,-112.956680,-123.263298,-107.799248,...,529.966,33,18,24014.0,NaN,8,605021;605027;605032;605085;605091;605097;6051...,-1.0,query,HeLa
9,-131.963470,-120.266258,100.880493,67.540001,-136.762527,31.225903,103.442215,-115.572365,-126.327248,-110.452354,...,534.201,45,27,39566.0,NaN,7,614048;614055;614062;614069;614075;614080;614085,129.0,query,HeLa


In [15]:
latent_space.shape

(1653858, 36)

In [16]:
latent_space.to_csv('peptideprotonet_embedding_space_90Kto20KsplitTrain_epoch300_featuresScaled_HeLa4Reps_200ng_100ms_noEvidenceTrain.csv')

#### Embedding query runs with reference data

In [ ]:
# data = pd.concat([query_allpeptides, evidence_train], sort=False, ignore_index=True)
evidence_train.reset_index(inplace=True, drop=True)


In [ ]:
data = evidence_train
data

In [ ]:
x = torch.from_numpy(data[attr_names].to_numpy()).float()
x = StandardScaler().fit_transform(x)
print(x.shape)
z = model(torch.from_numpy(x).float().to(device))
protonet_embedding_z = z.cpu().detach().numpy()

# append all meta-data
latent_dim = 10
latent_space = pd.DataFrame(protonet_embedding_z, columns = ['dim_'+str(i) for i in range(latent_dim)])


In [ ]:
ref_latent_space = pd.concat([latent_space, data], axis = 1)
ref_latent_space.head(15)

In [ ]:
ref_latent_space.to_csv('peptideprotonet_embedding_space_90Kto20KsplitTrain_epoch300_featuresScaled_EvidenceTrain.csv')

In [ ]:
#### plotting the embedding

In [ ]:
random.seed(614)
row_idx = np.random.choice(protonet_embedding_z.shape[0], 50000, replace=False)
z_subsample = protonet_embedding_z[row_idx]

In [ ]:
reducer = umap.UMAP(metric = 'cosine')
embedding = reducer.fit_transform(z_subsample)

In [ ]:
# color by charge - purple is charge 2
charge_state = query_allpeptides.Charge.astype("category")
charge_state = charge_state.to_numpy()
charge_sub = charge_state[row_idx]


plt.scatter(embedding[:,0], embedding[:,1],
           c=charge_sub, s = 0.1)
#plt.gca().set_aspect('equal', 'datalim')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
plt.colorbar()
plt.title('peptideprotonet latent space')

### Reload evidence and allPeptide tables and apply the model to them

### Old stuff start here

In [ ]:
evidence = pd.read_csv('PXD023049/txt/evidence.txt', sep='\t', engine='python', header=0)
print(evidence.shape)
allPeptide = pd.read_csv('PXD023049/txt/allPeptides.txt', sep='\t', engine='python', header=0)
print(allPeptide.shape)

allPeptide.rename({'Ion mobility index length': 'Ion mobility length'}, axis=1, inplace=True)

# df_diff = pd.concat([allPeptide[attrs], evidence[attrs]]).drop_duplicates(keep=False)
# print(df_diff.shape)




In [ ]:
# evidence.head()

In [ ]:
evidence = evidence.round({'Retention time': 1,  'Retention length':3})

In [ ]:
attrs = ['Raw file',
         'Charge',
         #'Mass', 
         #'m/z', 
#          'Retention time',
         #'Retention length',
         'Ion mobility index', 
         #'Ion mobility length',
         'MS/MS scan number',
         'Number of isotopic peaks',
         'Intensity'
        ]
df3 = pd.merge(evidence, allPeptide, on=attrs , how='outer', indicator='Exist')
#df3.head(15)

In [ ]:
df3.Exist.value_counts()

In [ ]:
df3

In [ ]:
### Apply the model to all allPeptide txt records except charge 1 idents
allPeptide = allPeptide.loc[(allPeptide['Intensity'] > 0) & \
                                (allPeptide['Charge'] != 1)]



allPeptide['study'] = 'query'
allPeptide['Species'] = 'Human'

attr_names = ['Charge','Mass', 'm/z', 'Retention time','Retention length',
              'Ion mobility index', 'Ion mobility length','Number of isotopic peaks']

In [ ]:
data = pd.concat([allPeptide, evidence_train], sort=False, ignore_index=True)
data

In [ ]:
x = torch.from_numpy(data[attr_names].to_numpy()).float()
x = StandardScaler().fit_transform(x)
print(x.shape)
z = model(torch.from_numpy(x).float().to(device))
protonet_embedding_z = z.cpu().detach().numpy()

# append all meta-data
latent_dim = 10
latent_space = pd.DataFrame(protonet_embedding_z, columns = ['dim_'+str(i) for i in range(latent_dim)])


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
ref_latent_space = pd.concat([latent_space, data], axis = 1)
ref_latent_space.head(15)

In [ ]:
ref_latent_space

In [ ]:
# ref_latent_space.to_csv('peptideprotonet_embedding_space_90Kto20KsplitTrain_epoch300_featuresScaled_tcells_cancer_query_icml2021.csv')

In [ ]:
ref_data = ref_latent_space[~ref_latent_space.study.isin(['query'])]
ref_embedding = ref_data.iloc[:,:10]
ref_labels = ref_data.PrecursorID
ref_run = ref_data['Raw file']

In [ ]:
query_data = ref_latent_space[ref_latent_space.study.isin(['query'])]
df3 = pd.merge(evidence, query_data, on=attrs , how='outer', indicator='Exist')

In [ ]:
df3.Exist.value_counts()

In [ ]:
# df3

In [ ]:
ident_query_embedding = df3.loc[df3.Exist == 'both','dim_0':'dim_9']
ident_query_labels = df3.loc[df3.Exist == 'both', 'Modified sequence_x'].astype(str).str.cat(df3.Charge[df3.Exist == 'both'].astype(str), sep='_')
ident_query_run = df3.loc[df3.Exist == 'both','Raw file']

In [ ]:
ident_query_labels = ident_query_labels.tolist()

In [ ]:
ref_labels = ref_labels.tolist()

In [ ]:
# these two need to be converted to numpy arrays using .to_numpy()
print(ident_query_embedding.shape)
print(len(ident_query_labels))
print(ref_embedding.shape)
print(len(ref_labels))

In [ ]:
embeddings = pd.concat([ref_embedding, ident_query_embedding]).to_numpy() # numpy array
labels = ref_labels + ident_query_labels # list

In [ ]:
print(embeddings.shape, len(labels))

In [ ]:
pdata = pd.concat([ref_embedding, ident_query_embedding])
pdata.reset_index(inplace=True, drop=True)

prun = pd.concat([ref_run, ident_query_run])
prun.reset_index(inplace=True, drop=True)


pd.concat([pdata,
          pd.DataFrame(labels, columns = ['labels']),
          prun], axis = 1)

In [ ]:
pd.concat([pdata,
          pd.DataFrame(labels, columns = ['labels']),
          prun], axis = 1).to_csv('reference_embeddings_and_metadata_icml2021.csv')

In [ ]:
query_embeddings = df3.loc[df3.Exist == 'right_only','dim_0':'dim_9']
query_embeddings

In [ ]:
query_metadata = df3.loc[df3.Exist == 'right_only', ['Raw file','Charge','Intensity', 'Isotope correlation']]
query_metadata

In [ ]:
pd.concat([query_embeddings, query_metadata], axis = 1).to_csv('query_embeddings_and_metadata_icml2021.csv')

In [ ]:
# query_embeddings = query_embeddings.to_numpy()